In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [2]:
from ipynb.fs.full.ClassificationPerformanceIndexes import classificationPerformanceIndexes, printClassificationPerformanceIndexes

Feature Classification

In [3]:
def TrainingKfold (X, train, test):
    X_train = X.iloc[train,:X.shape[1]-1]
    y_train = X.loc[train,'seizure']
    X_test = X.iloc[test,:X.shape[1]-1]
    y_test = X.loc[test,'seizure']
    return X_train, y_train, X_test, y_test

LSTM

In [4]:
def LstmModel (size, lstm_units, dense_units, dropout_percentage, loss_function, metric):
    model = Sequential()
    model.add(LSTM(lstm_units, recurrent_regularizer = l2(1e-2), activity_regularizer = l2(1e-4), bias_regularizer = l2(1e-6)))
    model.add(Dropout(dropout_percentage))
    model.add(Dense(dense_units, activation = 'relu', kernel_regularizer = l2(1e-3), bias_regularizer = l2(1e-2)))
    model.add(Dropout(dropout_percentage/2))
    model.add(Dense(1, activation = 'sigmoid', kernel_regularizer = l2(1e-3), bias_regularizer = l2(1e-2)))
    model.compile(optimizer = Adam(learning_rate = 1e-3), loss = loss_function, metrics = metric)
    return model

In [5]:
def LSTM_method (model, X_train, y_train, X_test, y_test, batch, epochs, results):
    print('Implementing LSTM...')
    start = time.time()
    es = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 5, mode = 'auto', restore_best_weights = True, verbose = 0)
    history = model.fit(X_train, y_train, batch_size = batch, epochs = epochs, validation_data = (X_test,y_test), callbacks = es, verbose = 0)
    lstm_ind = (model.predict(X_test, batch_size = batch) >= 0.5).astype('int')
    end = time.time()
    t = round(end - start,2)
    acc, snv, spc, ppv, f1, mcc, kappa, tt = classificationPerformanceIndexes (y_test, np.reshape(lstm_ind, lstm_ind.shape[0]), t)
    results.loc['LSTM', :] = acc, snv, spc, ppv, f1, mcc, kappa, t
    printClassificationPerformanceIndexes('LSTM', acc, snv, spc, ppv, f1, mcc, kappa)
    print('LSTM finished in', t,'sec\n')

In [6]:
def LSTM_method_Kfold(X, kf, cols, model, batch, epochs, results):
    f = pd.DataFrame(columns = cols)
    print('Implementing LSTM k-fold...')
    start = time.time()
    es = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 5, mode = 'auto', restore_best_weights = True, verbose = 0)
    for train, test in kf.split(X):
        X_train = X.iloc[train,:X.shape[1]-1]
        X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
        y_train = X.loc[train,'seizure'].values.astype(int)
        X_test = X.iloc[test,:X.shape[1]-1]
        X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))
        y_test = X.loc[test,'seizure'].values.astype(int)
        history = model.fit(X_train, y_train, batch_size = batch, epochs = epochs, validation_data = (X_test,y_test), callbacks = es, verbose = 0)
        lstm_ind = (model.predict(X_test, batch_size = batch) >= 0.5).astype('int')
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, np.reshape(lstm_ind, lstm_ind.shape[0]), 0)
    end = time.time()
    t = round(end - start,2)
    acc, snv, spc, ppv, f1, mcc, kappa, tt = np.array(f.mean(axis=0))
    results.loc['LSTM Kfold', :] = acc, snv, spc, ppv, f1, mcc, kappa, t
    printClassificationPerformanceIndexes('LSTM Kfold', acc, snv, spc, ppv, f1, mcc, kappa)
    print('LSTM finished in', t,'sec\n')

In [7]:
def CompleteLSTM (train_dat, test_dat, train_ind, test_ind, results, ft, kf, perfInd, epochs, batch, lstm_units, dense_units, dropout_percentage, loss_function, metric):
    X_train = np.reshape(train_dat.values, (train_dat.shape[0], 1, train_dat.shape[1]))
    y_train = train_ind.values.astype(int)
    X_test = np.reshape(test_dat.values, (test_dat.shape[0], 1, test_dat.shape[1]))
    y_test = test_ind.values.astype(int)

    lstm_model = LstmModel (train_dat.shape[1], lstm_units, dense_units, dropout_percentage, loss_function, metric)
    LSTM_method (lstm_model, X_train, y_train, X_test, y_test, batch, epochs, results)
    LSTM_method_Kfold (ft, kf, perfInd, lstm_model, batch, epochs, results)